### Based on this paper: https://www.jmlr.org/papers/volume3/bengio03a/bengio03a.pdf

In [1]:
from unicodedata import name
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt
import numpy as np

In [2]:
names = open("names.txt", "r").read().splitlines()

In [3]:
names[:10]

['emma',
 'olivia',
 'ava',
 'isabella',
 'sophia',
 'charlotte',
 'mia',
 'amelia',
 'harper',
 'evelyn']

In [7]:
chars = sorted(list(set(''.join(names))))
stoi = {s:i+1 for i,s in enumerate(chars)}
stoi['.'] = 0
itos = {i:s for s,i in stoi.items()}
print(stoi)

{'a': 1, 'b': 2, 'c': 3, 'd': 4, 'e': 5, 'f': 6, 'g': 7, 'h': 8, 'i': 9, 'j': 10, 'k': 11, 'l': 12, 'm': 13, 'n': 14, 'o': 15, 'p': 16, 'q': 17, 'r': 18, 's': 19, 't': 20, 'u': 21, 'v': 22, 'w': 23, 'x': 24, 'y': 25, 'z': 26, '.': 0}


In [17]:
block_size = 3 #context length of how many chars we take to predict the next char
X, Y = [],[]

for w in names[:5]:
    print(w)
    context = [0] * block_size
    for ch in w + ".":
        ix = stoi[ch]
        X.append(context)
        Y.append(ix)
        print(''.join(itos[i] for i in context), '--->', itos[ix])
        context = context[1:] + [ix] # rolling window effect, makes sense when you run it

X = torch.tensor(X)
Y = torch.tensor(Y)

emma
... ---> e
..e ---> m
.em ---> m
emm ---> a
mma ---> .
olivia
... ---> o
..o ---> l
.ol ---> i
oli ---> v
liv ---> i
ivi ---> a
via ---> .
ava
... ---> a
..a ---> v
.av ---> a
ava ---> .
isabella
... ---> i
..i ---> s
.is ---> a
isa ---> b
sab ---> e
abe ---> l
bel ---> l
ell ---> a
lla ---> .
sophia
... ---> s
..s ---> o
.so ---> p
sop ---> h
oph ---> i
phi ---> a
hia ---> .


In [38]:
X.shape, X.dtype, Y.shape, Y.dtype, X[1]

(torch.Size([32, 3]),
 torch.int64,
 torch.Size([32]),
 torch.int64,
 tensor([0, 0, 5]))

### Build the embedding look up table C <br />
in the paper, they embed 17,000 words into 30 dimension vectors. We can do something similar, but with our 27 possible chars, we can go smaller; like 2 dimensional maybe?

In [43]:
C = torch.randn((27,2))

In [48]:
C[X].shape

torch.Size([32, 3, 2])

In [36]:
X[13,2]

tensor(1)

In [46]:
C[X][13,2], X[13]

(tensor([-0.4839,  0.1449]), tensor([0, 0, 1]))

In [45]:
C[1]

tensor([-0.4839,  0.1449])

In [49]:
emb = C[X]
emb.shape

torch.Size([32, 3, 2])